In [3]:
import numpy as np
import pandas as pd
data = pd.read_csv("weatherX.csv")
data


,Outlook,Temp,Humidity,Windy,Play
0,Sunny,hot,high,False,no
1,Sunny,hot,high,True,yes
2,Overcast,hot,high,False,no
3,Rainy,mild,high,False,no
4,Rainy,cool,normal,False,yes
5,Rainy,cool,normal,True,no
6,Overcast,cool,normal,True,no
7,Sunny,mild,high,False,no
8,Sunny,cool,normal,False,yes
9,Rainy,mild,normal,False,yes


In [4]:
data.columns[-1]

'Play'

In [5]:
data.index

RangeIndex(start=0, stop=14, step=1)

In [6]:
data.iloc[:5][ ["Outlook", "Temp"] ] 


,Outlook,Temp
0,Sunny,hot
1,Sunny,hot
2,Overcast,hot
3,Rainy,mild
4,Rainy,cool


In [7]:
data.iloc[2:5][ ["Outlook", "Temp", "Play"] ] 

,Outlook,Temp,Play
2,Overcast,hot,no
3,Rainy,mild,no
4,Rainy,cool,yes


In [8]:
playCounts = data.groupby(['Play'])['Play'].count()
playCounts

Play
no     7
yes    7
Name: Play, dtype: int64

In [9]:
#import scipy.stats to help calculate the entropy of Play
import scipy.stats as stats
originalEntropy=stats.entropy(playCounts, base = 2)
originalEntropy

1.0

In [10]:
PlayGivenOutlook=data.groupby(['Play', 'Outlook'])['Play'].count()
PlayGivenOutlook

Play  Outlook 
no    Overcast    2
      Rainy       2
      Sunny       3
yes   Overcast    2
      Rainy       3
      Sunny       2
Name: Play, dtype: int64

In [11]:
PlayGivenOutlookDF = PlayGivenOutlook.unstack()
PlayGivenOutlookDF

Outlook,Overcast,Rainy,Sunny
Play,,,
no,2,2,3
yes,2,3,2


In [12]:
PlayGivenOutlookDF.sum()

Outlook
Overcast    4
Rainy       5
Sunny       5
dtype: int64

In [13]:
PlayGivenOutlookDF.sum().sum()

14

In [14]:
outlookDistribution=PlayGivenOutlookDF.sum() / PlayGivenOutlookDF.sum().sum()
outlookDistribution

Outlook
Overcast    0.285714
Rainy       0.357143
Sunny       0.357143
dtype: float64

In [15]:
#import scipy.stats to help calculate the entropy of Play when outlook is 'Overcast'
import scipy.stats as stats
entropyOvercast = stats.entropy(PlayGivenOutlookDF['Overcast'], base = 2)
entropyOvercast

1.0

In [16]:
#Calculate the entropy of Play when outlook is 'Rainy'
entropyRainy = stats.entropy(PlayGivenOutlookDF['Rainy'], base = 2)
entropyRainy

0.9709505944546688

In [17]:
#Calculate the entropy of Play when outlook is 'Sunny'
entropySunny = stats.entropy(PlayGivenOutlookDF['Sunny'], base = 2)
entropySunny

0.9709505944546688

In [18]:
#Calculate the entropy of Play when outlook is 'Overcast', 'Rainy', and 'Sunny' respectively
# and store the result in a list 
entropyGivenOutlookList = [stats.entropy(PlayGivenOutlookDF[outlook], base = 2) for outlook in PlayGivenOutlookDF.columns]
entropyGivenOutlookList

[1.0, 0.9709505944546688, 0.9709505944546688]

In [19]:
#Calculate the expected entropy of Play given outlook  
# based on entropyGivenOutlookList  and outlookDistribution
print("\n outlookDistribution\n", np.array(outlookDistribution))
print("\n entropyGivenOutlookList\n", np.array(entropyGivenOutlookList ))
print("\n np.array(entropyGivenOutlookList) * np.array(outlookDistribution)\n", 
      np.array(entropyGivenOutlookList) * np.array(outlookDistribution) )

entropyGivenOutlook = ( np.array(entropyGivenOutlookList) * np.array(outlookDistribution) ).sum()
print("\n entropyGivenOutlook=", entropyGivenOutlook)
print("Information gain given Outlook is", originalEntropy - entropyGivenOutlook)


 outlookDistribution
 [0.28571429 0.35714286 0.35714286]

 entropyGivenOutlookList
 [1.         0.97095059 0.97095059]

 np.array(entropyGivenOutlookList) * np.array(outlookDistribution)
 [0.28571429 0.34676807 0.34676807]

 entropyGivenOutlook= 0.9792504246104777
Information gain given Outlook is 0.020749575389522268


In [20]:
#Given a dta frame object containing a data set (last column represent the class attriubute such as 'Play')
# and a feature such as 'Outlook'
# determine and return a numpy array containing the information gain of each theother columns (features)

import scipy.stats as stats

def infoGainOfGivenFeature(data, feature):
    
    last = data.columns[-1]
    
    play_given_feature = data.groupby([last, feature])[last].count().unstack()
    play = data.groupby([last])[last].count()
    feature_distribution = play_given_feature.sum()/play_given_feature.sum().sum()
    play_given_feature_list = [stats.entropy(play_given_feature[x], base = 2) for x in play_given_feature.columns]
    
    entropy_given_feature = (np.array(feature_distribution) * np.array(play_given_feature_list)).sum()
    
    og_entropy = stats.entropy(play, base = 2)
    
    print(og_entropy - entropy_given_feature)
    
    
    
    
    


infoGainOfGivenFeature(data, 'Outlook')


0.020749575389522268


In [21]:
#Given a dta frame object containing a data set (last column represent the class attriubute such as 'Play')
# determine and return a numpy array containing the information gain of each theother columns (features)
def infoGain(data):
    print("\nInfo gain:\n")
    for feature in data.columns[ :-1]:
        print("given ", feature, infoGainOfGivenFeature(data, feature))
    

#Analyze the information gains of the features in the weather data set        
infoGain(data)


Info gain:

0.020749575389522268
given  Outlook None
0.0
given  Temp None
0.014771863965748477
given  Humidity None
0.0
given  Windy None


In [32]:
#********************************************************************************
#Create the data set as a dataframe object for predicting the actions of agent# 1
#********************************************************************************

import numpy as np
a = np.loadtxt("RPS_transcript1.txt", delimiter =',', usecols=range(0,3), dtype=np.int32)

#Data about the actions of the agent now and in the previous two matches
actionNow = a[2:, 1]
actionPrev = a[1:-1, 1]
actionPrevPrev = a[0:-2, 1]

#Data about the actions of the user in the previous two matches
uActionPrev = a[1:-1, 0]
uActionPrevPrev = a[0:-2, 0]

#Create the data set as a dataframe object for predicting the actions of agent# 1
import pandas as pd
data = pd.DataFrame({'1 userPrev':uActionPrev,  '2 userPrevPrev':uActionPrevPrev,
                     '3 agentPrev':actionPrev,  '4 agentPrevPrev':actionPrevPrev, '5 agentNow':actionNow})

data.head()
data.columns
data

,1 userPrev,2 userPrevPrev,3 agentPrev,4 agentPrevPrev,5 agentNow
0,2,2,2,0,0
1,0,2,0,2,2
2,1,0,2,0,2
3,2,1,2,2,1
4,1,2,1,2,2
5,1,1,2,1,2
6,2,1,2,2,0
7,0,2,0,2,1
8,0,0,1,0,1
9,0,0,1,1,2


In [33]:
#Analyze the information gains of the features in the data set constructued from Agent#1's transcript       
infoGain(data)


Info gain:

0.007346760155145349
given  1 userPrev None
0.007523157437527184
given  2 userPrevPrev None
0.0033979592086532673
given  3 agentPrev None
0.009671700186173027
given  4 agentPrevPrev None


In [37]:
#********************************************************************************
#Create the data set as a dataframe object for predicting the actions of agent# 2
#********************************************************************************

import numpy as np
a = np.loadtxt("RPS_transcript2.txt", delimiter =',', usecols=range(0,3), dtype=np.int32)

#Data about the actions of the agent now and in the previous two matches
actionNow = a[2:, 1]
actionPrev = a[1:-1, 1]
actionPrevPrev = a[0:-2, 1]

#Data about the actions of the user in the previous two matches
uActionPrev = a[1:-1, 0]
uActionPrevPrev = a[0:-2, 0]

#Create the data set as a dataframe object for predicting the actions of the agent
import pandas as pd
data = pd.DataFrame({'1 userPrev':uActionPrev,  '2 userPrevPrev':uActionPrevPrev,
                     '3 agentPrev':actionPrev,  '4 agentPrevPrev':actionPrevPrev, '5 agentNow':actionNow})
data.head()

print(data)

data.columns[-1]

     1 userPrev  2 userPrevPrev  3 agentPrev  4 agentPrevPrev  5 agentNow
0             2               1            1                0           2
1             1               2            2                1           2
2             2               1            2                2           2
3             1               2            2                2           0
4             0               1            0                2           1
5             1               0            1                0           2
6             0               1            2                1           0
7             2               0            0                2           2
8             1               2            2                0           0
9             2               1            0                2           1
10            0               2            1                0           2
11            2               0            2                1           1
12            0               2       

'5 agentNow'

In [36]:
#Analyze the information gains of the features in the data set constructued from Agent#2's transcript       
infoGain(data)

#Looking into agentPrev would give us the most information to what the agent would play next for Agent#2
#Looking int agentPrevPrev would give us the most information to what the agent would play next for Agent#1


Info gain:

0.001769034534315761
given  1 userPrev None
0.008091234979101802
given  2 userPrevPrev None
0.27860820563569577
given  3 agentPrev None
0.07651560455214446
given  4 agentPrevPrev None
